# Concurrency Primitives - Async Synchronization Tools

Lionherd provides async-first concurrency primitives built on AnyIO. These tools enable safe coordination between concurrent tasks:

**Core Primitives:**
- **Lock**: Mutual exclusion for critical sections
- **Semaphore**: Limit concurrent access to N resources
- **Event**: Signal completion or state changes across tasks
- **CapacityLimiter**: Fine-grained resource capacity management
- **Queue**: Producer-consumer communication
- **Condition**: Wait for complex conditions with notifications

**Key Features:**
- Context manager support (`async with`)
- AnyIO-compatible (works with asyncio, trio, etc.)
- Type-safe with generics (Queue)
- Statistics and observability

In [1]:
import asyncio

from lionherd_core.libs.concurrency import (
    CapacityLimiter,
    Condition,
    Event,
    Lock,
    Queue,
    Semaphore,
    create_task_group,
    sleep,
)

## 1. Lock - Mutual Exclusion

Lock ensures only one task accesses a critical section at a time. Essential for protecting shared state.

In [2]:
# Shared counter without protection (race condition)
counter = 0


async def increment_unsafe():
    global counter
    for _ in range(1000):
        # Read-modify-write is NOT atomic
        temp = counter
        await sleep(0)  # Simulate context switch
        counter = temp + 1


# Run 3 concurrent tasks
async def race_condition_demo():
    global counter
    counter = 0
    async with create_task_group() as tg:
        for _ in range(3):
            tg.start_soon(increment_unsafe)
    return counter


result = await race_condition_demo()
print(f"Unsafe counter (expected 3000): {result}")
print(f"Lost updates: {3000 - result}")

Unsafe counter (expected 3000): 1000
Lost updates: 2000


In [3]:
# Safe version with Lock
lock = Lock()
counter = 0


async def increment_safe():
    global counter
    for _ in range(1000):
        async with lock:  # Only one task in critical section
            temp = counter
            await sleep(0)
            counter = temp + 1


async def safe_counter_demo():
    global counter
    counter = 0
    async with create_task_group() as tg:
        for _ in range(3):
            tg.start_soon(increment_safe)
    return counter


result = await safe_counter_demo()
print(f"Safe counter: {result}")
print("✓ No lost updates!")

Safe counter: 3000
✓ No lost updates!


## 2. Semaphore - Limited Concurrency

Semaphore allows up to N tasks to access a resource simultaneously. Perfect for rate limiting or resource pools.

In [4]:
# Limit to 2 concurrent API calls
api_semaphore = Semaphore(2)


async def api_call(task_id: int):
    async with api_semaphore:
        print(f"Task {task_id} started (max 2 concurrent)")
        await sleep(0.5)  # Simulate API latency
        print(f"Task {task_id} finished")
        return f"Result {task_id}"


# Launch 5 tasks - only 2 run at once
async def semaphore_demo():
    # start_soon() returns None, so we use shared results list
    results = []

    async def wrapper(task_id: int):
        result = await api_call(task_id)
        results.append(result)

    async with create_task_group() as tg:
        for i in range(5):
            tg.start_soon(wrapper, i)

    return results


results = await semaphore_demo()
print(f"\nAll results: {results}")

Task 0 started (max 2 concurrent)
Task 1 started (max 2 concurrent)
Task 0 finished
Task 1 finished
Task 2 started (max 2 concurrent)
Task 3 started (max 2 concurrent)
Task 2 finished
Task 3 finished
Task 4 started (max 2 concurrent)
Task 4 finished

All results: ['Result 0', 'Result 1', 'Result 2', 'Result 3', 'Result 4']


## 3. Event - Cross-Task Signaling

Event allows tasks to wait for a signal from another task. Useful for initialization, checkpoints, or shutdown.

In [5]:
ready_event = Event()


async def initializer():
    print("Initializer: Loading resources...")
    await sleep(1)  # Simulate startup time
    print("Initializer: Ready!")
    ready_event.set()  # Signal waiting tasks


async def worker(worker_id: int):
    print(f"Worker {worker_id}: Waiting for initialization...")
    await ready_event.wait()  # Block until event is set
    print(f"Worker {worker_id}: Starting work!")
    await sleep(0.5)
    return f"Worker {worker_id} done"


async def event_demo():
    # start_soon() returns None, so we use shared results list
    results = []

    async def worker_wrapper(worker_id: int):
        result = await worker(worker_id)
        results.append(result)

    async with create_task_group() as tg:
        # Start initializer
        tg.start_soon(initializer)
        # Start workers (will wait for event)
        for i in range(3):
            tg.start_soon(worker_wrapper, i)

    return results


results = await event_demo()
print(f"\nResults: {results}")
print(f"Event is set: {ready_event.is_set()}")

Initializer: Loading resources...
Worker 0: Waiting for initialization...
Worker 1: Waiting for initialization...
Worker 2: Waiting for initialization...
Initializer: Ready!
Worker 0: Starting work!
Worker 1: Starting work!
Worker 2: Starting work!

Results: ['Worker 0 done', 'Worker 1 done', 'Worker 2 done']
Event is set: True


## 4. CapacityLimiter - Resource Management

CapacityLimiter manages integer capacity with fine-grained control. Track available/borrowed tokens and adjust limits dynamically.

In [6]:
# Create limiter with 3 total capacity
limiter = CapacityLimiter(3)

print("Initial state:")
print(f"  Total tokens: {limiter.total_tokens}")
print(f"  Available: {limiter.available_tokens}")
print(f"  Borrowed: {limiter.borrowed_tokens}")

Initial state:
  Total tokens: 3
  Available: 3
  Borrowed: 0


In [7]:
async def task_with_capacity(task_id: int):
    print(f"Task {task_id} waiting for capacity...")
    async with limiter:  # Acquire 1 token by default
        print(f"  Task {task_id} acquired (available: {limiter.available_tokens})")
        await sleep(0.3)
    print(f"  Task {task_id} released (available: {limiter.available_tokens})")


# Launch 5 tasks - only 3 tokens available
async def capacity_demo():
    async with create_task_group() as tg:
        for i in range(5):
            tg.start_soon(task_with_capacity, i)


await capacity_demo()
print(f"\nFinal state: {limiter.available_tokens}/{limiter.total_tokens} available")

Task 0 waiting for capacity...
Task 1 waiting for capacity...
Task 2 waiting for capacity...
Task 3 waiting for capacity...
Task 4 waiting for capacity...
  Task 0 acquired (available: 0)
  Task 1 acquired (available: 0)
  Task 2 acquired (available: 0)
  Task 0 released (available: 1)
  Task 1 released (available: 2)
  Task 2 released (available: 3)
  Task 3 acquired (available: 2)
  Task 4 acquired (available: 1)
  Task 3 released (available: 2)
  Task 4 released (available: 3)

Final state: 3/3 available


In [8]:
# Dynamic capacity adjustment
limiter.total_tokens = 5  # Increase capacity
print(f"Increased capacity to {limiter.total_tokens}")


# Borrower-tracked capacity
class Worker:
    def __init__(self, worker_id: int):
        self.id = worker_id


async def tracked_capacity_demo():
    worker = Worker(1)

    await limiter.acquire_on_behalf_of(worker)
    print(f"Worker {worker.id} acquired capacity")
    print(f"  Borrowed: {limiter.borrowed_tokens}")

    limiter.release_on_behalf_of(worker)
    print(f"Worker {worker.id} released capacity")
    print(f"  Borrowed: {limiter.borrowed_tokens}")


await tracked_capacity_demo()

Increased capacity to 5
Worker 1 acquired capacity
  Borrowed: 1
Worker 1 released capacity
  Borrowed: 0


## 5. Queue - Producer-Consumer Pattern

Queue enables safe communication between producer and consumer tasks. Supports backpressure via maxsize.

In [9]:
# Create queue with max 3 items
work_queue = Queue[str].with_maxsize(3)


async def producer(producer_id: int, count: int):
    for i in range(count):
        item = f"P{producer_id}-Item{i}"
        await work_queue.put(item)  # Blocks when queue is full
        print(f"Producer {producer_id} produced: {item}")
        await sleep(0.2)


async def consumer(consumer_id: int):
    while True:
        try:
            # Try to get with timeout
            item = await asyncio.wait_for(work_queue.get(), timeout=1.0)
            print(f"  Consumer {consumer_id} consumed: {item}")
            await sleep(0.3)  # Simulate processing
        except TimeoutError:
            print(f"  Consumer {consumer_id} timed out (queue empty)")
            break


async def queue_demo():
    async with create_task_group() as tg:
        # 2 producers, 2 consumers
        tg.start_soon(producer, 1, 3)
        tg.start_soon(producer, 2, 3)
        await sleep(0.1)  # Let producers start
        tg.start_soon(consumer, 1)
        tg.start_soon(consumer, 2)


await queue_demo()
await work_queue.close()

Producer 1 produced: P1-Item0
Producer 2 produced: P2-Item0
  Consumer 1 consumed: P1-Item0
  Consumer 2 consumed: P2-Item0
Producer 1 produced: P1-Item1
Producer 2 produced: P2-Item1
Producer 1 produced: P1-Item2
Producer 2 produced: P2-Item2
  Consumer 1 consumed: P1-Item1
  Consumer 2 consumed: P2-Item1
  Consumer 1 consumed: P1-Item2
  Consumer 2 consumed: P2-Item2
  Consumer 1 timed out (queue empty)
  Consumer 2 timed out (queue empty)


## 6. Condition - Coordinated Waiting

Condition allows tasks to wait for complex conditions and be notified when state changes. More flexible than Event.

In [10]:
# Shared state protected by condition
condition = Condition()
data_ready = False
data_value = None


async def data_producer():
    global data_ready, data_value
    await sleep(1)  # Simulate data collection

    async with condition:
        data_value = "Important Data"
        data_ready = True
        print("Producer: Data ready, notifying all waiters")
        condition.notify_all()  # Wake all waiting tasks


async def data_consumer(consumer_id: int):
    print(f"Consumer {consumer_id}: Waiting for data...")
    async with condition:
        while not data_ready:  # Wait for condition
            await condition.wait()
        print(f"Consumer {consumer_id}: Got data: {data_value}")
        return data_value


async def condition_demo():
    global data_ready, data_value
    data_ready = False
    data_value = None

    # start_soon() returns None, so we use shared results list
    results = []

    async def consumer_wrapper(consumer_id: int):
        result = await data_consumer(consumer_id)
        results.append(result)

    async with create_task_group() as tg:
        # Start consumers first (will wait)
        for i in range(3):
            tg.start_soon(consumer_wrapper, i)
        # Start producer
        tg.start_soon(data_producer)

    return results


results = await condition_demo()
print(f"\nAll consumers got: {results}")

Consumer 0: Waiting for data...
Consumer 1: Waiting for data...
Consumer 2: Waiting for data...
Producer: Data ready, notifying all waiters
Consumer 0: Got data: Important Data
Consumer 1: Got data: Important Data
Consumer 2: Got data: Important Data

All consumers got: ['Important Data', 'Important Data', 'Important Data']


## 7. Statistics and Observability

Event and Condition provide statistics for monitoring and debugging.

In [11]:
# Create event with multiple waiters
monitored_event = Event()


async def monitored_waiter(waiter_id: int):
    await monitored_event.wait()
    return waiter_id


async def statistics_demo():
    # Start waiters
    async with create_task_group() as tg:
        # start_soon() returns None, results not needed here
        for i in range(5):
            tg.start_soon(monitored_waiter, i)

        # Check statistics before setting
        await sleep(0.1)
        stats_before = monitored_event.statistics()
        print("Before set:")
        print(f"  Tasks waiting: {stats_before.tasks_waiting}")

        # Set event
        monitored_event.set()

    # After all waiters complete
    stats_after = monitored_event.statistics()
    print("\nAfter set:")
    print(f"  Tasks waiting: {stats_after.tasks_waiting}")
    print(f"  Event is set: {monitored_event.is_set()}")


await statistics_demo()

Before set:
  Tasks waiting: 5

After set:
  Tasks waiting: 0
  Event is set: True


## 8. Real-World Pattern: Bounded Task Executor

Combine primitives for a practical pattern: execute tasks with bounded concurrency and progress tracking.

In [12]:
class BoundedExecutor:
    def __init__(self, max_concurrent: int):
        self.semaphore = Semaphore(max_concurrent)
        self.lock = Lock()
        self.completed = 0
        self.total = 0

    async def execute(self, coro):
        async with self.semaphore:  # Limit concurrency
            result = await coro
            async with self.lock:  # Protect counter
                self.completed += 1
                print(f"Progress: {self.completed}/{self.total}")
            return result

    async def map(self, tasks):
        self.total = len(tasks)
        self.completed = 0

        # start_soon() returns None, so we use shared results list
        results = []

        async def wrapper(task):
            result = await self.execute(task)
            results.append(result)

        async with create_task_group() as tg:
            for task in tasks:
                tg.start_soon(wrapper, task)

        return results


# Use executor
async def slow_task(task_id: int):
    await sleep(0.3)
    return f"Task {task_id} result"


executor = BoundedExecutor(max_concurrent=3)
tasks = [slow_task(i) for i in range(10)]
results = await executor.map(tasks)
print(f"\nAll results: {len(results)} tasks completed")

Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10

All results: 10 tasks completed


## Summary Checklist

**Concurrency Primitives:**
- ✅ **Lock**: Mutual exclusion for critical sections (race condition protection)
- ✅ **Semaphore**: Limit N concurrent operations (rate limiting, resource pools)
- ✅ **Event**: Signal state changes across tasks (initialization, checkpoints)
- ✅ **CapacityLimiter**: Fine-grained capacity management (fractional tokens, dynamic limits)
- ✅ **Queue**: Producer-consumer communication (backpressure via maxsize)
- ✅ **Condition**: Wait for complex conditions (coordinated state changes)

**Best Practices:**
- ✅ Use `async with` for automatic acquire/release
- ✅ Lock protects shared state, Semaphore limits access
- ✅ Event for one-time signals, Condition for repeated notifications
- ✅ Queue maxsize provides backpressure (prevents memory issues)
- ✅ Monitor statistics for debugging and observability
- ✅ Combine primitives for complex patterns (executor example)

**Common Patterns:**
- Lock + counter: Safe shared state
- Semaphore: API rate limiting, connection pools
- Event: Startup coordination, shutdown signals
- Queue: Task distribution, pipeline processing
- Condition: Producer-consumer with state checks

**Next Steps:**
- See `TaskGroup` for structured concurrency
- See `ExecutorPool` for parallel execution
- See AnyIO docs for cross-library compatibility